In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[6],4
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,1e-24
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[3]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312        1 -4.773080e-06                    1 -6.464328e-05
0.000750        2 -4.283837e-06                    2 -1.728518e-04
0.001052        3 -3.083034e-06                    3 -1.948306e-05
0.001476        4 -1.523695e-06                    4 -1.354087e-05
0.002070        5  4.413248e-07                    5 -9.379646e-06
0.002904        6  2.888268e-06                    6 -6.143829e-06
0.004074        7  5.902320e-06                    7 -3.994816e-06
0.005714        8  9.582776e-06                    8 -1.050715e-06
0.008015        9  1.404460e-05                    9  1.683122e-06
0.011243       10  1.942057e-05                   10  4.624074e-06
0.015771       11  2.586223e-05                   11  7.871716e-06
0.022122       12  3.354533e-05                   12  1.163725e-05
0.031031       13  4.262456e-05                   13  1.749080e-05
0.043528       14  5.201964e-05                   14  2.236056e-05
0.061057       15  6.025742e-05                   15  2.477350e-05
0.085645       16  6.813618e-05                   16  2.763930e-05
0.120136       17  7.675202e-05                   17  3.143748e-05
0.168516       18  8.620178e-05                   18  3.547512e-05
0.236378       19  9.655763e-05                   19  3.968554e-05
0.331549       20  1.078836e-04                   20  4.422306e-05
0.465100       21  1.189622e-04                   21  4.986973e-05
0.652400       22  1.102238e-04                   22  4.678471e-05
0.915100       23  7.974316e-05                   23  3.304035e-05
1.283650       24  5.169624e-05                   24  2.064212e-05
1.800600       25  3.173575e-05                   25  1.192714e-05
2.525700       26  1.777655e-05                   26  5.872251e-06
3.542800       27  8.236996e-06                   27  1.733586e-06
4.969550       28  2.558256e-06                   28 -6.987135e-07
6.970850       29 -3.924401e-07                   29 -1.940268e-06
9.778100       30 -2.199722e-06                   30 -2.718749e-06
13.715850      31 -3.543780e-06                   31 -3.318903e-06
19.239350      32 -4.462989e-06                   32 -3.763040e-06
26.987250      33 -4.727486e-06                   33 -3.919027e-06
37.855300      34 -4.405250e-06                   34 -3.793903e-06
53.100050      35 -3.915309e-06                   35 -3.728387e-06
73.887500      36 -3.378088e-06                   36 -3.649289e-06
97.662500      37 -2.959760e-06                   37 -3.527507e-06
121.437500     38 -2.890764e-06                   38 -3.558496e-06
145.212500     39 -4.144496e-06                   39 -5.014048e-06
168.987500     40 -5.653625e-06                   40 -6.713295e-06
192.762500     41 -7.514925e-06                   41 -8.793861e-06
216.537500     42 -9.167296e-06                   42 -1.070771e-05
240.312500     43 -1.091409e-05                   43 -1.278059e-05
264.087500     44 -1.302050e-05                   44 -1.521783e-05
287.862500     45 -1.513232e-05                   45 -1.778785e-05
311.637500     46 -1.431798e-05                   46 -1.852775e-05
335.412500     47 -6.277209e-06                   47 -1.431678e-05
359.187500     48  1.085454e-05                   48 -3.934272e-06
382.962500     49  4.840762e-05                   49  1.757395e-05
406.737500     50  1.335205e-04                   50  7.061360e-05
430.512500     51  2.921810e-04                   51  1.763259e-04
454.287500     52  5.562243e-04                   52  3.623848e-04
478.062500     53  9.577298e-04                   53  6.604383e-04
501.837500     54  1.525780e-03                   54  1.101927e-03
525.612500     55  2.281277e-03                   55  1.694393e-03
549.387500     56  3.230963e-03                   56  2.466436e-03
573.162500     57  4.367963e-03                   57  3.42556

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -32.825730  0.000000e+00 -32.825730 -32.832910   
0.000624    2     -32.825730  6.857989e-10 -32.825730 -32.832910   
0.000876    3     -32.825730  9.835365e-10 -32.825730 -32.832910   
0.001229    4     -32.825730  1.469212e-09 -32.825730 -32.832910   
0.001723    5     -32.825730  2.262727e-09 -32.825730 -32.832910   
0.002417    6     -32.825730  3.558040e-09 -32.825730 -32.832910   
0.003391    7     -32.825730  5.671116e-09 -32.825730 -32.832910   
0.004757    8     -32.825730  9.115577e-09 -32.825730 -32.832910   
0.006672    9     -32.825730  1.472571e-08 -32.825730 -32.832910   
0.009359    10    -32.825730  2.385628e-08 -32.825730 -32.832910   
0.013128    11    -32.825730  3.870486e-08 -32.825730 -32.832910   
0.018415    12    -32.825730  6.283407e-08 -32.825730 -32.832910   
0.025830    13    -32.825730  1.020134e-07 -32.825730 -32.832910   
0.036232    14    -32.825730  1.655388e-07 -32.825730 -32.832911   
0.050823    15    -32.825730  2.667541e-07 -32.825730 -32.832911   
0.071291    16    -32.825730  4.233210e-07 -32.825729 -32.832911   
0.100000    17    -32.825730  6.622101e-07 -32.825729 -32.832911   
0.140271    18    -32.825730  1.026468e-06 -32.825729 -32.832911   
0.196760    19    -32.825730  1.581729e-06 -32.825728 -32.832911   
0.275997    20    -32.825730  2.427866e-06 -32.825727 -32.832911   
0.387100    21    -32.825730  3.716306e-06 -32.825726 -32.832911   
0.543100    22    -32.825729  5.664122e-06 -32.825724 -32.832911   
0.761700    23    -32.825729  8.240720e-06 -32.825721 -32.832911   
1.068500    24    -32.825729  1.109341e-05 -32.825718 -32.832911   
1.498800    25    -32.825729  1.406823e-05 -32.825715 -32.832911   
2.102400    26    -32.825730  1.716015e-05 -32.825713 -32.832911   
2.949000    27    -32.825733  2.036421e-05 -32.825711 -32.832912   
4.136600    28    -32.825735  2.368084e-05 -32.825710 -32.832914   
5.802500    29    -32.825738  2.725302e-05 -32.825710 -32.832915   
8.139200    30    -32.825742  3.140312e-05 -32.825710 -32.832918   
11.417000   31    -32.825748  3.642775e-05 -32.825711 -32.832921   
16.014700   32    -32.825756  4.258411e-05 -32.825713 -32.832926   
22.464000   33    -32.825767  5.030446e-05 -32.825716 -32.832933   
31.510500   34    -32.825782  6.081029e-05 -32.825722 -32.832943   
44.200100   35    -32.825804  7.640459e-05 -32.825728 -32.832957   
62.000000   36    -32.825836  9.998174e-05 -32.825736 -32.832978   
85.775000   37    -32.825880  1.338970e-04 -32.825746 -32.833008   
109.550000  38    -32.825924  1.697463e-04 -32.825754 -32.833040   
133.325000  39    -32.825969  2.067277e-04 -32.825762 -32.833073   
157.100000  40    -32.826032  2.581524e-04 -32.825774 -32.833120   
180.875000  41    -32.826115  3.260259e-04 -32.825790 -32.833182   
204.650000  42    -32.826224  4.134908e-04 -32.825812 -32.833264   
228.425000  43    -32.826354  5.169134e-04 -32.825837 -32.833364   
252.200000  44    -32.826504  6.365838e-04 -32.825867 -32.833482   
275.975000  45    -32.826680  7.756137e-04 -32.825904 -32.833623   
299.750000  46    -32.826886  9.385803e-04 -32.825947 -32.833790   
323.525000  47    -32.827129  1.141417e-03 -32.825987 -32.833990   
347.300000  48    -32.827421  1.415853e-03 -32.826004 -32.834230   
371.075000  49    -32.827771  1.797204e-03 -32.825974 -32.834519   
394.850000  50    -32.828298  2.460197e-03 -32.825838 -32.834958   
418.625000  51    -32.829124  3.662415e-03 -32.825462 -32.835652   
442.400000  52    -32.830335  5.696166e-03 -32.824639 -32.836667   
466.175000  53    -32.832007  8.934789e-03 -32.823073 -32.838077   
489.950000  54    -32.834195  1.381948e-02 -32.820375 -32.839933   
513.725000  55    -32.836921  2.084257e-02 -32.816078 -32.842265   
537.500000  56    -32.840170  3.051688e-02 -32.809654 -32.845032   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')